In [1]:
%pip install --upgrade --quiet langchain-pinecone langchain-openai langchain pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.


In [1]:

%pip install python-dotenv langchain langchain-openai langchain-community langchain-text-splitters docx2txt langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [3]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [9]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
pinecone_api_key = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents([], embedding, index_name=index_name)

batch_size = 100

for i in range(0, len(document_list), batch_size):
    batch = document_list[i: i+batch_size]
    database.add_documents(batch)

In [16]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query)

In [18]:
retrieved_docs

[Document(id='8341928c-297b-48da-b9dd-882989aa58cd', metadata={'source': './tax.docx'}, page_content='1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n\n다. 대통령령으로 정하는 농어가부업소득\n\n라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n\n마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n\n바. 대통령령으로 정하는 작물재배업에서 발생하는 소득\n\n사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n\n3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 대통령령으로 정하는 복무 중인 병(兵)이 받는 급여\n\n나. 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n\n다. 「산업재해보상보험법」에 따라 수급권자가 받는 요양급여, 휴업급여, 장해급여, 간병급여, 유족급여, 유족특별급여, 장해특별급여, 장의비 또는 근로의 제공으로 인한 부상ㆍ질병ㆍ사망과 관련하여 근로자나 그 유족이 받는 배상ㆍ보상 또는 위자(慰藉)의 성질이 있는 급여\n\n라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 요양보상금, 휴업보상금

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [12]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

c:\Users\1\miniconda3\envs\disRagEnv\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [17]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [14]:
ai_message = qa_chain({"query": query})

C:\Users\1\AppData\Local\Temp\ipykernel_31112\3455095564.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query": query})


In [15]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '죄송합니다. 주어진 문맥으로는 연봉 5천만 원인 직장인의 소득세를 계산할 수 없습니다. 소득세 계산에는 세율, 공제항목 등 추가적인 정보가 필요합니다.'}